In [5]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "cat_test")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'infer_merge'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-d13'
# aml_compute_target = 'vdsm-cluster-l80'
aml_compute = AmlCompute(ws, aml_compute_target)

env = Environment.get(workspace=ws,name="vdsm_all",version="1")

In [9]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="infer_merge.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        run_invocation_timeout=2400,
                                        # node_count=3)
                                        node_count=1)

month_pipeline_param = PipelineParameter(name="month", default_value="2022-01")

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

# merge_step = ParallelRunStep(
#     name="merge_for_inference",
#     inputs=ds_list,
#     arguments=[
#         "--month", month_pipeline_param
#     ],
#     parallel_run_config=pipeline_run_config)

trigger_step = PythonScriptStep(
    name="trigger_fusion_pipeline",
    script_name="trigger_fusion_pipeline.py",
    compute_target=aml_compute,
    source_directory="."
    )

step_sequence = StepSequence(steps=[merge_step, trigger_step])  

pipeline = Pipeline(workspace=ws, steps=step_sequence)
# pipeline_run = exp.submit(pipeline)
# print('VDSM training data fusion submitted for execution')
# RunDetails(pipeline_run).show()
# pipeline_run.wait_for_completion()

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
pipeline.publish(name="infer_merge", description="Merge S2 data for inference", version=8)

Created step merge_for_inference [d9f34e6a][a274a6ab-4197-4e20-afaa-38f83dd295b5], (This step will run and generate new outputs)
Created step trigger_fusion_pipeline [b628afec][56fe2fec-6e5b-4f4b-9155-36d0e318a8ce], (This step will run and generate new outputs)


Pipeline(Name: infer_merge,
Id: f6e2f88a-905a-4a9e-b256-f8e94f105f26,
Status: Active,
Endpoint: https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/f6e2f88a-905a-4a9e-b256-f8e94f105f26)